# Abfragen zur Ausgabe relationaler Daten als XML

[Basic syntax of the FOR XML clause](https:\learn.microsoft.com\en-us\sql\relational-databases\xml\basic-syntax-of-the-for-xml-clause?view=sql-server-ver16)

## FOR XML RAW

Ergebnis: KundenBestellungen-Bsp1.xml

In [ ]:
-- RAW macht Elemente für Zeilen und XML-Attribute für Spalten
SELECT Kunde.KundenNr, Firma, Bestelldatum, CAST(SUM(Menge * Verkaufspreis) AS DECIMAL(9,2)) AS Rechnungsbetrag
FROM Kunde
JOIN Bestellung ON Kunde.KundenNr = Bestellung.KundenNr
JOIN BestellDetail ON Bestellung.BestellungID = BestellDetail.BestellungID
WHERE Kunde.KundenNr <= 3
AND YEAR(Bestelldatum) = 2018
GROUP BY Kunde.KundenNr, Firma, Bestelldatum
ORDER BY Kunde.KundenNr, Bestelldatum
FOR XML RAW, ROOT('KundenBestellungen');

## FOR XML AUTO

Ergebnis: KundenBestellungen-Bsp2.xml

In [ ]:
-- FOR XML AUTO erstellt Elemente je Tabelle
-- und XML-Attribute für Spalten, entsprechend verschachtelt
SELECT Kunde.KundenNr, Firma, Bestelldatum, CAST(SUM(Menge * Verkaufspreis)AS DECIMAL(9,2)) AS Rechnungsbetrag
FROM Kunde
JOIN Bestellung ON Kunde.KundenNr = Bestellung.KundenNr
JOIN BestellDetail ON Bestellung.BestellungID = BestellDetail.BestellungID
WHERE Kunde.KundenNr <= 3
AND YEAR(Bestelldatum) = 2018
GROUP BY Kunde.KundenNr, Firma, Bestelldatum
ORDER BY Kunde.KundenNr, Bestelldatum
FOR XML AUTO, ROOT('KundenBestellungen');

## FOR XML AUTO, ELEMENTS

Ergebnis: KundenBestellungen-Bsp3.xml

In [ ]:
-- Version mit Elementen für alles
SELECT Kunde.KundenNr, Firma, Bestelldatum
    , CAST(SUM(Menge * Verkaufspreis) AS DECIMAL(9,2)) AS Rechnungsbetrag
FROM Kunde
JOIN Bestellung ON Kunde.KundenNr = Bestellung.KundenNr
JOIN BestellDetail ON Bestellung.BestellungID = BestellDetail.BestellungID
WHERE Kunde.KundenNr <= 3
AND YEAR(Bestelldatum) = 2018
GROUP BY Kunde.KundenNr, Firma, Bestelldatum
ORDER BY Kunde.KundenNr, Bestelldatum
FOR XML AUTO, ELEMENTS, ROOT('KundenBestellungen');

## FOR XML PATH für flexible Mischung von XML-Elementen und Attributen

Ergebnis: KundenBestellungen-Bsp4.xml

In [ ]:
-- FOR XML PATH zur Mischung von XML-Elementen und Attributen
SELECT Kunde.KundenNr AS "@KundenNr", Firma, Bestelldatum AS "Bestellung/@Bestelldatum"
    , CAST(SUM(Menge * Verkaufspreis) AS DECIMAL(9,2)) AS "Bestellung/@Rechnungsbetrag"
FROM Kunde
JOIN Bestellung ON Kunde.KundenNr = Bestellung.KundenNr
JOIN BestellDetail ON Bestellung.BestellungID = BestellDetail.BestellungID
WHERE Kunde.KundenNr <= 3
AND YEAR(Bestelldatum) = 2018
GROUP BY Kunde.KundenNr, Firma, Bestelldatum
ORDER BY Kunde.KundenNr, Bestelldatum
FOR XML PATH('Kunde'), ROOT('KundenBestellungen');

In [ ]:
-- BAUSTELLE: Ausgabe der Unterabfragen noch nicht gelöst
SELECT Kunde.KundenNr, Kunde.Firma
    , (
        SELECT Bestelldatum
            , CAST(SUM(Menge * Verkaufspreis) AS DECIMAL(9,2)) AS Rechnungssumme
            , (
               SELECT Produkt, Menge, CAST(Verkaufspreis AS DECIMAL(9,2)) AS Verkaufspreis
               FROM Produkt
               JOIN BestellDetail AS Details ON Produkt.ProduktID = Details.ProduktID
               AND Details.BestellungID = Bestellung.BestellungID 
               FOR XML AUTO, TYPE
            ) AS Positionen
        FROM Bestellung 
        JOIN BestellDetail ON Bestellung.BestellungID = BestellDetail.BestellungID
        WHERE Kunde.KundenNr = Bestellung.KundenNr
        AND YEAR(Bestelldatum) = 2018
        GROUP BY Bestellung.BestellungID, Bestelldatum
        FOR XML AUTO, TYPE
     ) AS Bestellungen
FROM Kunde 
WHERE KundenNr <= 3
FOR XML AUTO, ELEMENTS;